# Underground Music Recommendator

Aidan Schneider

## Introduction

This project inplements a Retrieval Augmented Generation (RAG) system designed to recommend underground music based on user preferences. The system uses a combination of techniques to provide semantically rich and contextually aware music recommendations.

The pipeline begins with data preparation. A CSV dataset was created containing album information that was scraped from bandcamp.com. The CSV is then processed by an embedding model to generate dense vector embeddings that represent the semantic meaning of each album. These embeddings and the original data are then stored in a Hugging Face Datasets format.

Similarity search is achieved by indexing the embeddigns with FAISS. This allows for rapid retrieval of the most relevant albums based on the user's query. Then those albums are reranked based on a calculated score value before being passed into the LLM

Once the relevant documents are received the user's prompt as well as the document data are all given to the LLM which is what the user is directly interacting with.

In [1]:
from main import rag, music_chat
from embed_albums import MODEL_ID, get_text_embeddings, embed, faiss_index, get_model_id, set_model_id
from datasets import load_from_disk
import os
import faiss

/home/soot/anaconda3/envs/ug-music/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiemts

In this notebook I will be doing variations of what I already have in `main.py` which I consider to be the main pipeline. As I continue these experiments I will keep in the components that I consider better.

### Embedding

In the main pipeline I use `textembedding-gecko@003` for all the embeddings. Here I will be using `textembedding-gecko-multilingual@001` since there are probably albums with languages that are not english.

Setting up the new embedding model.

In [2]:
model_id = 'textembedding-gecko-multilingual@001'

set_model_id(model_id)

get_model_id()

'textembedding-gecko-multilingual@001'

Creates the new embeddings

In [5]:
if not os.path.exists('./bandcamp_data_ml_embeddings'):
    dataset = load_from_disk('bandcamp_data')
    dataset.map(embed, batched=True, batch_size=32)
    dataset.save_to_disk('bandcamp_data_ml_embeddings')

Loads the embeddings into a faiss index

In [6]:
dataset = load_from_disk('bandcamp_data_ml_embeddings')

faiss_index(dataset, 'faiss_ml_index.index')

Indexing Embeddings


Generates a response with the given components

In [4]:
def generate_response(prompt, index_path, data_path, sysprompt_path, model_id):
    user_prompt = prompt
    index = faiss.read_index(index_path)
    dataset = load_from_disk(data_path)

    system_prompt = ''

    with open(sysprompt_path, 'r') as f:
        for line in f:
            system_prompt += line

    music_chat(model_id, dataset, index, system_prompt, user_prompt=user_prompt, chat=False)

The response with the multilingual embeddings.

In [5]:
user_prompt = 'Dark and gloomy music from Canada'
index_path = 'faiss_ml_index.index'
data_path = 'bandcamp_data_ml_embeddings'
sysprompt_path = 'SYSPROMPT.txt'
model_id = 'gemini-2.0-flash'

generate_response(user_prompt, index_path, data_path, sysprompt_path, model_id)

Based on the provided data, here are some examples of dark and gloomy music from Canada:

*   **Artist:** DIM
    *   **Genres:** black metal, christian, medieval ambient, soundtrack, ambient, choral, dark ambient, dark folk, dungeon synth, fantasy, fantasy synth, holy synth, medieval synth, neoclassical darkwave
    *   **Description:** "dim's steeped sky, stained light plucks us up from the shady grove or dungeon grim of previous entries and sets us high amongst the firmament. all praise and glory to the most high." "listening to this album is like stepping into a lonely sanctuary that feels both distant and familiar."
*   **Artist:** into the pale abyss
    *   **Album:** "the darkness within" EP
    *   **Genres:** dark techno, EBM, electronic, cyberpunk, darksynth, darkwave, horrorsynth, midtempo, techno, witch house
    *   **Description:** "this non-stop consecration of auditory defilement is just what the witch doctor ordered.", "ritualistic madness music for witches and cultis

The same prompt and model using the english embeddings.

In [6]:
user_prompt = 'Dark and gloomy music from Canada'
index_path = 'faiss_index.index'
data_path = 'bandcamp_data_embeddings'
sysprompt_path = 'SYSPROMPT.txt'
model_id = 'gemini-2.0-flash'

generate_response(user_prompt, index_path, data_path, sysprompt_path, model_id)

Based on the provided data, here are some music suggestions that fit the "dark and gloomy music from Canada" criteria:

*   **Dim:** This artist is tagged with "dark ambient," "Canada," and the reviews describe the music as stepping into a "lonely sanctuary," "quiet and sacred space," and having a "dark and gloomy" sound, suggesting a fit for the prompt.
*   **"The darkness within" EP by \[Artist Name]:** This release is tagged with "dark techno," "darkwave," "darksynth," "EBM," and "Canada." The reviews also mention "auditory defilement," "ritualistic madness," and a "dark and vivid universe," aligning with the "dark and gloomy" theme.
*   **Brandon Wars:** This artist is tagged with "dark ambient," "drone," and "Canada," and the review describes "complex cold wave tunes" and "ambient accompaniment to this long winter," implying a potentially gloomy atmosphere.


Overall, I prefer the multilingual embeddings for this specific prompt since over different tests it tends to give better responses. This makes sense since there are presumably many albums with non-english names.

### LLM

In the main pipeline I have been using Gemini 2.0 Flash. For this experiment I want to try the Gemini 1.5 Pro model since it's a little bigger and may fit my use case better.

The 1.5 Pro model

In [7]:
user_prompt = 'Dark and gloomy music from Canada'
index_path = 'faiss_ml_index.index'
data_path = 'bandcamp_data_ml_embeddings'
sysprompt_path = 'SYSPROMPT.txt'
model_id = 'gemini-1.5-pro-002'

generate_response(user_prompt, index_path, data_path, sysprompt_path, model_id)

The artist **dim** from Canada makes dark and gloomy music often tagged as dark ambient, dungeon synth, and fantasy synth.  Their music is frequently described with words like "beautiful," "sacred," "transcendent," and "healing," but also includes elements of dissonance and darkness. This aligns with the request for dark and gloomy while acknowledging the specific aesthetic described in the context.  Other Canadian artists mentioned don't seem to fit the "dark and gloomy" criteria as closely, though "The Darkness Within" EP by Blood & Opium has a darker, heavier electronic/industrial sound.  Brandon Wars also has some darker elements, but is described as "cold wave" and ambient, which might not be precisely what you're looking for.



Comparing this output with the baseline from above (The english embeddings) I would say that the 1.5 pro model definitely explains things better. However, this is at the cost of time which is almost double that of the flash model. I will be sticking with the flash model.

### System Prompt

The final experiment I will be running is changing the system prompt. I have noticed that whenever I restart the main program the format of the response varies greatly between sessions. The goal of this experiment is to make the responses a bit more uniform between sessions.

I specifically added this to the system prompt:

6. **Response Format:**
   * Make sure to list every album you are given
   * For each album give the artist(s) and a short description of the album from the album data given to you
   * If an album is not a great match to the user's preferences make sure to let the user know why
   * You should always list all 5 albums given to you

In [8]:
user_prompt = 'Dark and gloomy music from Canada'
index_path = 'faiss_ml_index.index'
data_path = 'bandcamp_data_ml_embeddings'
sysprompt_path = 'SYSPROMPT2.txt'
model_id = 'gemini-2.0-flash'

generate_response(user_prompt, index_path, data_path, sysprompt_path, model_id)

Based on the provided information, here are some Canadian artists creating dark and gloomy music:

*   **Dim:** This artist creates music in the dark ambient, dungeon synth, fantasy, fantasy synth, holy synth, medieval synth, and neoclassical darkwave genres. The music is described as beautiful, transcendent, and capable of transporting the listener to a lonely sanctuary.
*   **The Darkness Within:** This project falls into the dark techno, e.b.m, electronic, cyberpunk, darksynth, darkwave, ebm, electro, electronica, horrorsynth, midtempo, techno, and witch house categories. The music is described as ritualistic, dark, and intense.
*   **Brandon Wars:** This artist explores electronic, dark ambient, drone, electronica, pop rock, and soundscapes. One reviewer noted the "engagingly complex cold wave tunes" and the "perfect ambient accompaniment to this long winter."


In [9]:
user_prompt = 'Dark and gloomy music from Canada'
index_path = 'faiss_ml_index.index'
data_path = 'bandcamp_data_ml_embeddings'
sysprompt_path = 'SYSPROMPT.txt'
model_id = 'gemini-2.0-flash'

generate_response(user_prompt, index_path, data_path, sysprompt_path, model_id)

Based on the provided data, here are some options for dark and gloomy music from Canada:

*   **Dim:** Described as "dark ambient" and "dungeon synth", and associated with the phrases: "steeped sky, stained light dim," "tolling iron bells," "hued sunbeam." The reviews indicate a sound that can be dark, atmospheric, and contemplative. The tags "neoclassical darkwave", "dark folk" also suggest a gloomy element.

*   **"The darkness within" EP by (artist name not explicitly provided):** Categorized as "dark techno," "darksynth," "darkwave," "E.B.M.," and "witch house." The reviews use terms like "auditory defilement" and "ritualistic madness," indicating a very dark and intense sound.

*   **Brandon Wars:** Tagged as "dark ambient" and "drone". One review mentioned that one side of the album is "ambient accompaniment to this long winter", which also evokes a gloomy imagery.

It's worth noting that musical taste is subjective, so exploring these options will be the best way to determine if

While this didn't do exactly what I wanted I appreciate the deeper descriptions that come out of the new system prompt.